In [ ]:
# Appriori
import numpy as np
from python_utils import *
from itertools import combinations, chain

def count_occurences(itemset, Transactions):
  count = 0
  for i in range(len(Transactions)):
    if set(itemset).issubset(set(Transactions[i])):
      count += 1
  return count
def join_two_itemsets(it1, it2, order):
  it1.sort(key = lambda x: order.index(x))
  it2.sort(key = lambda x: order.index(x))
  for i in range(len(it1) - 1):
    if it1[i] != it2[i]:
      return []
  if order.index(it1[-1]) < order.index(it2[-1]):
    return it1 + [it2[-1]]

  return []

def join_set_itemsets(set_of_its, order):
  C = []
  for i in range(len(set_of_its)):
    for j in range(i+1, len(set_of_its)):
      it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
      if len(it_out) > 0:
        C.append(it_out)
  return C

def get_frequent(itemsets, Transactions, min_support, prev_discarded):
  L = []
  supp_count = []
  new_discarded = []
  
  k = len(prev_discarded.keys())

  for s in range(len(itemsets)):
    discarded_before = False
    if k > 0:
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemsets[s])):
          discarded_before = True
          break

    if not discarded_before:
      count = count_occurences(itemsets[s], Transactions)
      if count/len(Transactions) >= min_support:
        L.append(itemsets[s])
        supp_count.append(count)
      else:
        new_discarded.append(itemsets[s])
  return L , supp_count, new_discarded
  
def load_transactions(path_to_data, order):
  Transactions = []
  with open(path_to_data, 'r') as fid:
    for lines in fid:
      str_line = list(lines.strip().split(','))
      _t = list(np.unique(str_line))
      _t.sort(key = lambda x: order.index(x))
      Transactions.append(_t)
  return Transactions

def print_table(T, supp_count):
  print("Itemset | Frequency")
  for k in range(len(T)):
    print("{} : {}".format(T[k], supp_count[k]))
  print("\n\n")

import csv
Transactions = []
with open('/content/drive/MyDrive/Pamir/tdb.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

for i in range(len(data)):
  Transactions.append(data[i])


num_trans = len(Transactions)
# Transactions
# Transactions
orders = []
orders.append(Transactions[0][0])
for i in range(0,len(Transactions)):
  for j in range(len(Transactions[i])):
    count = 0
    k = 0
    while(k < len(orders)):
      if Transactions[i][j] == orders[k]:
        count += 1
        break
      k += 1
    if count == 0:
      orders.append(Transactions[i][j])

order = sorted(orders)

C = {}
L = {}
itemset_size = 1
Discarded = { itemset_size : []}
C.update({itemset_size : [ [f] for f in order]})

min_support = 1/5
min_confidence = 1
supp_count_L = {}
f, sup, new_discarded = get_frequent(C[itemset_size], Transactions, min_support, Discarded)
Discarded.update({itemset_size : new_discarded})
L.update({itemset_size : f})
supp_count_L.update({itemset_size : sup})

print_table(L[1], supp_count_L[1])
k = itemset_size + 1
convergence = False
while not convergence:
  C.update({ k: join_set_itemsets(L[k-1], order)})
  print("Table C{}:\n".format(k))
  print_table(C[k], [count_occurences(it, Transactions) for it in C[k]])
  f,sup, new_discarded = get_frequent(C[k], Transactions, min_support,Discarded)
  Discarded.update({k : new_discarded})
  L.update({k : f})
  supp_count_L.update({k : sup})
  if len(L[k]) == 0:
    convergence = True
  else:
    print("Table L{}:\n".format(k))
    print_table(L[k], supp_count_L[k])
  k += 1

def powerset(s):
  return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s) + i)))

def write_rules(X, X_S, S, conf, supp, num_trans):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
  out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  
  return out_rules


assoc_rules_str = ""
for i in range(1, len(L)):
  for j in range(len(L[i])):
    s = powerset(L[i][j])
    s.pop()
    for z in s:
      S = set(z)
      X = set(L[i][j])
      X_S = set(X-S)
      sup_x = count_occurences(X , Transactions)
      sup_x_s = count_occurences(X_S, Transactions)
      conf = sup_x / count_occurences(S, Transactions)
      if conf >= min_confidence and sup_x >= min_support:
        assoc_rules_str += write_rules(X, X_S, S, conf, sup_x,num_trans)

print("-----------------------Association Rule-------------------\n")
print(assoc_rules_str)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
order = ['I' + str(i) for i in range(1,6)]
print(order)

['I1', 'I2', 'I3', 'I4', 'I5']


In [ ]:
import csv
Transactions = []
with open('/content/drive/MyDrive/Pamir/tdb.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

for i in range(len(data)):
  Transactions.append(sorted(data[i]))


orders = []
orders.append(Transactions[0][0])
for i in range(0,len(Transactions)):
  for j in range(len(Transactions[i])):
    count = 0
    k = 0
    while(k < len(orders)):
      if Transactions[i][j] == orders[k]:
        count += 1
        break
      k += 1
    if count == 0:
      orders.append(Transactions[i][j])

orders = sorted(orders)
print(len(orders))

11


In [ ]:
import numpy as np
from python_utils import *
from itertools import combinations, chain

def count_occurences(itemset, Transactions):
  count = 0
  for i in range(len(Transactions)):
    if set(itemset).issubset(set(Transactions[i])):
      count += 1
  return count

def get_frequent(itemsets, Transactions):
  L = []
  supp_count = []
  new_discarded = []
  num_trans = len(Transactions)
  # k = len(prev_discarded.keys())

  for s in range(len(itemsets)):
    count = count_occurences(itemsets[s], Transactions)
    
    L.append(itemsets[s])
    supp_count.append(count)
    # else:
    #   new_discarded.append(itemsets[s])
  return L , supp_count
  
def load_transactions(path_to_data, order):
  Transactions = []
  with open(path_to_data, 'r') as fid:
    for lines in fid:
      str_line = list(lines.strip().split(','))
      _t = list(np.unique(str_line))
      _t.sort(key = lambda x: order.index(x))
      Transactions.append(_t)
  return Transactions

def print_PIS_table(T, supp_count, PIS):
  print("Itemset|Frequency|PIS")
  for k in range(len(T)):
    print("{} : {} : {}".format(T[k], supp_count[k], PIS[k]))
  print("\n\n")

def print_MIS_table(T, supp_count,supp_count_L, Dict):
  print("Itemset                | Frequency | MIS")
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
  print("\n\n")

arr2 = []

def print_Exact_table(T, supp_count,supp_count_L, Dict):
  print("Exact Itemsets")
  print("-------------------------------------------")
  print("Itemset                | Frequency | MIS")
  count = 0
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    if(supp_count[k] == min(arr)):
      arr2.append(T[k])
      print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
      count += 1
  if(count ==  0):
        print("There is no exact itemsets")
  print("\n\n")


def join_two_itemsets(it1, it2, order):
  it1.sort(key = lambda x: order.index(x))
  it2.sort(key = lambda x: order.index(x))

  for i in range(len(it1) - 1):
    if it1[i] != it2[i]:
      return []
  if order.index(it1[-1]) < order.index(it2[-1]):
    return it1 + [it2[-1]]

  return []

def join_set_itemsets(set_of_its, order):
  C = []
  for i in range(len(set_of_its)):
    for j in range(i+1, len(set_of_its)):
      it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
      if len(it_out) > 0:
        C.append(it_out)
  return C

path_to_data = "/content/drive/MyDrive/Pamir/data.txt"
order = ['I' + str(i) for i in range(1,6)]
Transactions = load_transactions(path_to_data,order)
# Transactions
C = {}
L = {}
min_conf = 1
itemset_size = 1
num_trans = len(Transactions)
# Discarded = { itemset_size : []}
C.update({itemset_size : [ [f] for f in order]})
C

supp_count_L = {}
f, sup = get_frequent(C[itemset_size], Transactions)
# Discarded.update({itemset_size : new_discarded})
L.update({itemset_size : f})
supp_count_L.update({itemset_size : sup})

print(len(C[itemset_size]))
PIS = []
Dict = {}
for i in range(len(C[itemset_size])):
  n = int(input("Enter item PIS: "))
  PIS.append(n)

# List is converting into string  
def convertList(list1):  
  
    return ' '.join([str(e) for e in list1]) #List comprehension  

for i in range(len(L[1])):
    # case = {L[1][i] : supp_count_L[1][i]}
    case = {convertList(L[1][i]) : supp_count_L[1][i]}
    Dict.update(case)
# print(Dict)
print_PIS_table(L[1], supp_count_L[1], PIS)

k = itemset_size + 1
convergence = False
while not convergence:
  arr = []
  C.update({ k: join_set_itemsets(L[k-1], order)})
  print("Table C{}:\n".format(k))
  print_MIS_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
  print_Exact_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
  f,sup = get_frequent(C[k], Transactions)
  L.update({k : f})
  
  supp_count_L.update({k : sup})
  if len(L[k]) == 1:
    convergence = True
  else:
    print()
  k += 1

def powerset(s):
  return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s) + i)))

def write_rules(X, X_S, S, conf, supp, num_trans):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
  # out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  
  return out_rules
assoc_rules_str = ""
for i in range(len(arr2)):
  # for j in range(len(arr2[i])):
  s = list(powerset(arr2[i]))
  s.pop()
  for z in s:
    S = set(z)
    X = set(arr2[i])
    X_S = set(X-S)
    sup_x = count_occurences(X , Transactions)
    sup_x_s = count_occurences(X_S, Transactions)
    conf = sup_x / count_occurences(S, Transactions)
    if conf >= min_conf:
      assoc_rules_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_str)

In [ ]:
# Exact rule mining and apriori algorithm

import csv
import numpy as np
from python_utils import *
from itertools import combinations, chain

def count_occurences(itemset, Transactions):
  count = 0
  for i in range(len(Transactions)):
    if set(itemset).issubset(set(Transactions[i])):
      count += 1
  return count

def get_frequent_apriori(itemsets, Transactions, min_support, prev_discarded):
  L = []
  supp_count = []
  new_discarded = []
  
  k = len(prev_discarded.keys())

  for s in range(len(itemsets)):
    discarded_before = False
    if k > 0:
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemsets[s])):
          discarded_before = True
          break

    if not discarded_before:
      count = count_occurences(itemsets[s], Transactions)
      if count/len(Transactions) >= min_support:
        L.append(itemsets[s])
        supp_count.append(count)
      else:
        new_discarded.append(itemsets[s])
  return L , supp_count, new_discarded

def get_frequent_exact(itemsets, Transactions):
  L = []
  supp_count = []
  new_discarded = []
  num_trans = len(Transactions)

  for s in range(len(itemsets)):
    count = count_occurences(itemsets[s], Transactions)
    
    L.append(itemsets[s])
    supp_count.append(count)
  return L , supp_count

def load_transactions(path_to_data, order):
    Transactions = []
    with open(path_to_data, 'r') as fid:
      for lines in fid:
        str_line = list(lines.strip().split(','))
        _t = list(np.unique(str_line))
        _t.sort(key = lambda x: order.index(x))
        Transactions.append(_t)
    return Transactions

def print_PIS_table(T, supp_count, PIS):
  print("Itemset|Frequency|PIS")
  for k in range(len(T)):
    if(supp_count[k] <= PIS[k]):
      print("{} : {} : {}".format(T[k], supp_count[k], PIS[k]))
  print("\n\n")

def print_MIS_table(T, supp_count,supp_count_L, Dict):
  print("Itemset                | Frequency | MIS")
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
  print("\n\n")

arr2 = []

def print_Exact_table(T, supp_count,supp_count_L, Dict):
  print("Exact Itemsets")
  print("-------------------------------------------")
  print("Itemset                | Frequency | MIS")
  count = 0
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    if(supp_count[k] == min(arr)):
      arr2.append(T[k])
      print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
      count += 1
  if(count ==  0):
        print("There is no exact itemsets")
  print("\n\n")

def print_table(T, supp_count):
    print("Itemset | Frequency")
    for k in range(len(T)):
      print("{} : {}".format(T[k], supp_count[k]))
    print("\n\n")

def join_two_itemsets(it1, it2, order):
    it1.sort(key = lambda x: order.index(x))
    it2.sort(key = lambda x: order.index(x))

    for i in range(len(it1) - 1):
      if it1[i] != it2[i]:
        return []
    if order.index(it1[-1]) < order.index(it2[-1]):
      return it1 + [it2[-1]]

    return []

def join_set_itemsets(set_of_its, order):
    C = []
    for i in range(len(set_of_its)):
      for j in range(i+1, len(set_of_its)):
        it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
        if len(it_out) > 0:
          C.append(it_out)
    return C

def powerset(s):
  return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s) + i)))


def write_rules(X, X_S, S, conf, supp, num_trans):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
#   out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  
  return out_rules

Transactions = []
# /content/drive/MyDrive/Pamir/tdb - Copy.csv
with open('/content/drive/MyDrive/Pamir/tdb - Copy.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

for i in range(len(data)):
  Transactions.append(data[i])

num_trans = len(Transactions)
# Transactions
# Transactions
orders = []
orders.append(Transactions[0][0])
for i in range(0,len(Transactions)):
  for j in range(len(Transactions[i])):
    count = 0
    k = 0
    while(k < len(orders)):
      if Transactions[i][j] == orders[k]:
        count += 1
        break
      k += 1
    if count == 0:
      orders.append(Transactions[i][j])

order = sorted(orders)

C = {}
L = {}
itemset_size = 1
num_trans = len(Transactions)
supp_count_L = {}
C.update({itemset_size : [ [f] for f in order]})

assoc_rules_apriori_str = ""
assoc_rules_exact_str = ""
def aprioriAlgo(assoc_rules_apriori_str):
    min_support = 5/10
    min_confidence = 1
    Discarded = { itemset_size : []}
    f, sup, new_discarded = get_frequent_apriori(C[itemset_size], Transactions, min_support, Discarded)
    Discarded.update({itemset_size : new_discarded})
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})
    print_table(L[1], supp_count_L[1])
    k = itemset_size + 1
    convergence = False
    while not convergence:
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_table(C[k], [count_occurences(it, Transactions) for it in C[k]])
      f,sup, new_discarded = get_frequent_apriori(C[k], Transactions, min_support,Discarded)
      Discarded.update({k : new_discarded})
      L.update({k : f})
      supp_count_L.update({k : sup})
      if len(L[k]) == 0:
          convergence = True
      else:
          print("Table L{}:\n".format(k))
          print_table(L[k], supp_count_L[k])
      k += 1

    for i in range(1,len(L)):
      for j in range(len(L[i])):
        s = powerset(L[i][j])
        s.pop()
        for z in s:
          S = set(z)
          X = set(L[i][j])
          X_S = set(X-S)
          sup_x = count_occurences(X , Transactions)
          sup_x_s = count_occurences(X_S, Transactions)
          conf = sup_x / count_occurences(S, Transactions)
          if conf >= min_confidence and sup_x >= min_support:
              assoc_rules_apriori_str += write_rules(X, X_S, S, conf, sup_x,num_trans)

    return assoc_rules_apriori_str

def exactRule(assoc_rules_exact_str):
    assoc_rules_str = ""
    min_conf = 1
    f, sup = get_frequent_exact(C[itemset_size], Transactions)
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})
    PIS = []
    Dict = {}
    for i in range(len(C[itemset_size])):
      n = int(input("Enter item PIS: "))
      PIS.append(n)

    # List is converting into string  
    def convertList(list1):  
    
        return ' '.join([str(e) for e in list1]) #List comprehension  

    for i in range(len(L[1])):
        # case = {L[1][i] : supp_count_L[1][i]}
        case = {convertList(L[1][i]) : supp_count_L[1][i]}
        Dict.update(case)
    # print(Dict)
    
    print_PIS_table(L[1], supp_count_L[1], PIS)

    k = itemset_size + 1
    convergence = False
    while not convergence:
      arr = []
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_MIS_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      print_Exact_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      f,sup = get_frequent_exact(C[k], Transactions)
      L.update({k : f})
      
      supp_count_L.update({k : sup})
      if len(L[k]) == 1:
          convergence = True
      else:
          print()
      k += 1

    # for i in range(1, len(L)):
    for i in range(len(arr2)):
      # for j in range(len(arr2[i])):
      s = list(powerset(arr2[i]))
      s.pop()
      for z in s:
        S = set(z)
        X = set(arr2[i])
        X_S = set(X-S)
        sup_x = count_occurences(X , Transactions)
        sup_x_s = count_occurences(X_S, Transactions)
        conf = sup_x / count_occurences(S, Transactions)
        if conf >= min_conf:
          assoc_rules_exact_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
          
    return assoc_rules_exact_str
print("***********************Rule mining using apriori algorithm***********************")
assoc_rules_apriori_str = aprioriAlgo(assoc_rules_apriori_str)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_apriori_str)
print("***********************Rule mining using exact mining***********************")
assoc_rules_exact_str = exactRule(assoc_rules_exact_str)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_exact_str)

In [ ]:
# Exact rule mining and apriori algorithm

import csv
import numpy as np
from python_utils import *
from itertools import combinations, chain

def count_occurences(itemset, Transactions):
  count = 0
  for i in range(len(Transactions)):
    if set(itemset).issubset(set(Transactions[i])):
      count += 1
  return count

def get_frequent_apriori(itemsets, Transactions, min_support, prev_discarded):
  L = []
  supp_count = []
  new_discarded = []
  
  k = len(prev_discarded.keys())

  for s in range(len(itemsets)):
    discarded_before = False
    if k > 0:
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemsets[s])):
          discarded_before = True
          break

    if not discarded_before:
      count = count_occurences(itemsets[s], Transactions)
      if count/len(Transactions) >= min_support:
        L.append(itemsets[s])
        supp_count.append(count)
      else:
        new_discarded.append(itemsets[s])
  return L , supp_count, new_discarded

def get_frequent_exact(itemsets, Transactions):
  L = []
  supp_count = []
  new_discarded = []
  num_trans = len(Transactions)

  for s in range(len(itemsets)):
    count = count_occurences(itemsets[s], Transactions)
    
    L.append(itemsets[s])
    supp_count.append(count)
  return L , supp_count

def load_transactions(path_to_data, order):
    Transactions = []
    with open(path_to_data, 'r') as fid:
      for lines in fid:
        str_line = list(lines.strip().split(','))
        _t = list(np.unique(str_line))
        _t.sort(key = lambda x: order.index(x))
        Transactions.append(_t)
    return Transactions

def print_PIS_table(T, supp_count, PIS):
  print("Itemset|Frequency|PIS")
  for k in range(len(T)):
    # if(supp_count[k] <= PIS[k]):
    print("{} : {} : {}".format(T[k], supp_count[k], PIS[k]))
  print("\n\n")

def print_MIS_table(T, supp_count,supp_count_L, Dict):
  print("Itemset                | Frequency | MIS")
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
  print("\n\n")

arr2 = []

def print_Exact_table(T, supp_count,supp_count_L, Dict):
  print("Exact Itemsets")
  print("-------------------------------------------")
  print("Itemset                | Frequency | MIS")
  count = 0
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    if(supp_count[k] == min(arr)):
      arr2.append(T[k])
      print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
      count += 1
  if(count ==  0):
        print("There is no exact itemsets")
  print("\n\n")

def print_table(T, supp_count):
    print("Itemset | Frequency")
    for k in range(len(T)):
      print("{} : {}".format(T[k], supp_count[k]))
    print("\n\n")

def join_two_itemsets(it1, it2, order):
    it1.sort(key = lambda x: order.index(x))
    it2.sort(key = lambda x: order.index(x))

    for i in range(len(it1) - 1):
      if it1[i] != it2[i]:
        return []
    if order.index(it1[-1]) < order.index(it2[-1]):
      return it1 + [it2[-1]]

    return []

def join_set_itemsets(set_of_its, order):
    C = []
    for i in range(len(set_of_its)):
      for j in range(i+1, len(set_of_its)):
        it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
        if len(it_out) > 0:
          C.append(it_out)
    return C

def powerset(s):
  return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s) + i)))


def write_rules(X, X_S, S, conf, supp, num_trans):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
#   out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  
  return out_rules

Transactions = []
# /content/drive/MyDrive/Pamir/tdb - Copy.csv
with open('/content/drive/MyDrive/Pamir/tdb.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

for i in range(len(data)):
  Transactions.append(data[i])

num_trans = len(Transactions)
# Transactions
# Transactions
orders = []
orders.append(Transactions[0][0])
for i in range(0,len(Transactions)):
  for j in range(len(Transactions[i])):
    count = 0
    k = 0
    while(k < len(orders)):
      if Transactions[i][j] == orders[k]:
        count += 1
        break
      k += 1
    if count == 0:
      orders.append(Transactions[i][j])

order = sorted(orders)

C = {}
L = {}
itemset_size = 1
num_trans = len(Transactions)
supp_count_L = {}
C.update({itemset_size : [ [f] for f in order]})

assoc_rules_apriori_str = ""
assoc_rules_exact_str = ""
def aprioriAlgo(assoc_rules_apriori_str):
    min_support = 2/5
    min_confidence = 1
    Discarded = { itemset_size : []}
    f, sup, new_discarded = get_frequent_apriori(C[itemset_size], Transactions, min_support, Discarded)
    Discarded.update({itemset_size : new_discarded})
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})
    print_table(L[1], supp_count_L[1])
    k = itemset_size + 1
    convergence = False
    while not convergence:
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_table(C[k], [count_occurences(it, Transactions) for it in C[k]])
      f,sup, new_discarded = get_frequent_apriori(C[k], Transactions, min_support,Discarded)
      Discarded.update({k : new_discarded})
      L.update({k : f})
      supp_count_L.update({k : sup})
      if len(L[k]) == 0:
          convergence = True
      else:
          print("Table L{}:\n".format(k))
          print_table(L[k], supp_count_L[k])
      k += 1

    for i in range(1,len(L)):
      for j in range(len(L[i])):
        s = powerset(L[i][j])
        s.pop()
        for z in s:
          S = set(z)
          X = set(L[i][j])
          X_S = set(X-S)
          sup_x = count_occurences(X , Transactions)
          sup_x_s = count_occurences(X_S, Transactions)
          conf = sup_x / count_occurences(S, Transactions)
          if conf >= min_confidence and sup_x >= min_support:
              assoc_rules_apriori_str += write_rules(X, X_S, S, conf, sup_x,num_trans)

    return assoc_rules_apriori_str

def exactRule(assoc_rules_exact_str):
    assoc_rules_str = ""
    min_conf = 1
    PIS1 = []
    PIS = []
    Dict = {}
    f1, sup1 = get_frequent_exact(C[itemset_size], Transactions)
    for i in range(len(C[itemset_size])):
      n = int(input("Enter item PIS: "))
      PIS1.append(n)
    f = []
    sup = []
    for i in range(0,len(PIS1)):
      if(sup1[i] < PIS1[i]):
        f.append(f1[i])
        sup.append(sup1[i])
        PIS.append(PIS1[i])
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})  
    # L.update({itemset_size : f})
    # supp_count_L.update({itemset_size : sup})
    # print(L)
    # print(supp_count_L)
    # List is converting into string  
    def convertList(list1):  
    
        return ' '.join([str(e) for e in list1]) #List comprehension  

    for i in range(len(L[1])):
        # case = {L[1][i] : supp_count_L[1][i]}
        case = {convertList(L[1][i]) : supp_count_L[1][i]}
        Dict.update(case)
    # print(Dict)
    
    print_PIS_table(L[1], supp_count_L[1], PIS)

    k = itemset_size + 1
    convergence = False
    while not convergence:
      arr = []
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_MIS_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      print_Exact_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      f,sup = get_frequent_exact(C[k], Transactions)
      L.update({k : f})
      
      supp_count_L.update({k : sup})
      if len(L[k]) == 1:
          convergence = True
      else:
          print()
      k += 1

    # for i in range(1, len(L)):
    for i in range(len(arr2)):
      # for j in range(len(arr2[i])):
      s = list(powerset(arr2[i]))
      s.pop()
      for z in s:
        S = set(z)
        X = set(arr2[i])
        X_S = set(X-S)
        sup_x = count_occurences(X , Transactions)
        sup_x_s = count_occurences(X_S, Transactions)
        conf = sup_x / count_occurences(S, Transactions)
        if conf >= min_conf:
          assoc_rules_exact_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
          
    return assoc_rules_exact_str
print("***********************Rule mining using apriori algorithm***********************")
assoc_rules_apriori_str = aprioriAlgo(assoc_rules_apriori_str)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_apriori_str)
print("***********************Rule mining using exact mining***********************")
assoc_rules_exact_str = exactRule(assoc_rules_exact_str)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_exact_str)

In [ ]:
f = open("/content/drive/MyDrive/Pamir/PIS_tdb.txt", "r")
arr = []
for x in f:
  arr.append(x)

print(arr)

['0\n', '0\n', '1\n', '2\n', '5\n', '6\n', '7\n', '8\n', '9\n', '0\n', '0']


In [ ]:
with open('/content/drive/MyDrive/Pamir/PIS_tdb.txt', 'r') as file:
    all_file = file.read().strip()  # Read and remove any extra new line
    all_file_list = all_file.split('\n')  # make a list of lines
    final_data = [int(each_int) for each_int in all_file_list]
    print(final_data)

[0, 0, 1, 2, 5, 6, 7, 8, 9, 0, 0]


In [ ]:
# Final project
import csv
import numpy as np
from python_utils import *
from itertools import combinations, chain
import random
import time

def count_occurences(itemset, Transactions):
  count = 0
  for i in range(len(Transactions)):
    if set(itemset).issubset(set(Transactions[i])):
      count += 1
  return count

def get_frequent_apriori(itemsets, Transactions, min_support, prev_discarded):
  L = []
  supp_count = []
  new_discarded = []
  
  k = len(prev_discarded.keys())

  for s in range(len(itemsets)):
    discarded_before = False
    if k > 0:
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemsets[s])):
          discarded_before = True
          break

    if not discarded_before:
      count = count_occurences(itemsets[s], Transactions)
      if count/len(Transactions) >= min_support:
        L.append(itemsets[s])
        supp_count.append(count)
      else:
        new_discarded.append(itemsets[s])
  return L , supp_count, new_discarded

def get_frequent_exact(itemsets, Transactions):
  L = []
  supp_count = []
  new_discarded = []
  num_trans = len(Transactions)

  for s in range(len(itemsets)):
    count = count_occurences(itemsets[s], Transactions)
    
    L.append(itemsets[s])
    supp_count.append(count)
  return L , supp_count

def load_transactions(path_to_data, order):
    Transactions = []
    with open(path_to_data, 'r') as fid:
      for lines in fid:
        str_line = list(lines.strip().split(','))
        _t = list(np.unique(str_line))
        _t.sort(key = lambda x: order.index(x))
        Transactions.append(_t)
    return Transactions

def print_PIS_table(T, supp_count, PIS):
  print("Itemset|Frequency|PIS")
  for k in range(len(T)):
    print("{} : {} : {}".format(T[k], supp_count[k], PIS[k]))
  print("\n\n")

def print_MIS_table(T, supp_count,supp_count_L, Dict):
  print("Itemset                | Frequency | MIS")
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
  print("\n\n")

arr2 = []

def print_Exact_table(T, supp_count,supp_count_L, Dict):
  print("Exact Itemsets")
  print("-------------------------------------------")
  print("Itemset                | Frequency | MIS")
  count = 0
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    if(supp_count[k] == min(arr)):
      arr2.append(T[k])
      print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
      count += 1
  if(count ==  0):
        print("There is no exact itemsets")
  print("\n\n")

def print_table(T, supp_count):
    print("Itemset | Frequency")
    for k in range(len(T)):
      print("{} : {}".format(T[k], supp_count[k]))
    print("\n\n")

def join_two_itemsets(it1, it2, order):
    it1.sort(key = lambda x: order.index(x))
    it2.sort(key = lambda x: order.index(x))

    for i in range(len(it1) - 1):
      if it1[i] != it2[i]:
        return []
    if order.index(it1[-1]) < order.index(it2[-1]):
      return it1 + [it2[-1]]

    return []

def join_set_itemsets(set_of_its, order):
    C = []
    for i in range(len(set_of_its)):
      for j in range(i+1, len(set_of_its)):
        it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
        if len(it_out) > 0:
          C.append(it_out)
    return C

def powerset(s):
  return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s) + i)))


def write_rules(X, X_S, S, conf, supp, num_trans):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
  # out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  
  return out_rules

def write_rules_lift(X, X_S, S, conf, fre,num_trans):
  out_rules = ""
  lift = conf / fre
  if(lift >= 0.25):
    out_rules = ""
    out_rules += "Freq. Itemset: {}\n".format(X)
    out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
    out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
    out_rules += "     Lift: {0:2.3f} \n\n".format(lift)
    # out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  return out_rules

Transactions = []
filePath = input("Enter file path: ")
with open(filePath, newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

for i in range(len(data)):
  Transactions.append(data[i])

num_trans = len(Transactions)
# Transactions
# Transactions
orders = []
orders.append(Transactions[0][0])
for i in range(0,len(Transactions)):
  for j in range(len(Transactions[i])):
    count = 0
    k = 0
    while(k < len(orders)):
      if Transactions[i][j] == orders[k]:
        count += 1
        break
      k += 1
    if count == 0:
      orders.append(Transactions[i][j])

order = sorted(orders)
print(len(orders))
C = {}
L = {}
itemset_size = 1
num_trans = len(Transactions)
supp_count_L = {}
C.update({itemset_size : [ [f] for f in order]})

assoc_rules_apriori_str = ""
assoc_rules_apriori_str_lift = ""
assoc_rules_exact_str = ""
assoc_rules_apriori_str_exact_lift = ""
def aprioriAlgo(assoc_rules_apriori_str,assoc_rules_apriori_str_lift):
    timeTaking = time.time()
    min_support = float(input("Enter minimum support: "))
    min_confidence = 1
    Discarded = { itemset_size : []}
    f, sup, new_discarded = get_frequent_apriori(C[itemset_size], Transactions, min_support, Discarded)
    Discarded.update({itemset_size : new_discarded})
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})
    print_table(L[1], supp_count_L[1])
    k = itemset_size + 1
    convergence = False
    while not convergence:
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_table(C[k], [count_occurences(it, Transactions) for it in C[k]])
      f,sup, new_discarded = get_frequent_apriori(C[k], Transactions, min_support,Discarded)
      Discarded.update({k : new_discarded})
      L.update({k : f})
      supp_count_L.update({k : sup})
      if len(L[k]) == 0:
          convergence = True
      else:
          print("Table L{}:\n".format(k))
          print_table(L[k], supp_count_L[k])
      k += 1
    for i in range(1,len(L)):
      for j in range(len(L[i])):
        s = powerset(L[i][j])
        s.pop()
        for z in s:
          S = set(z)
          X = set(L[i][j])
          X_S = set(X-S)
          sup_x = count_occurences(X , Transactions)
          sup_x_s = count_occurences(X_S, Transactions)
          conf = sup_x / count_occurences(S, Transactions)
          if conf >= min_confidence and sup_x >= min_support:
              assoc_rules_apriori_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
              assoc_rules_apriori_str_lift += write_rules_lift(X, X_S, S, conf, sup_x_s,num_trans)
    
    timeTaking1 = time.time()
    totalTime = timeTaking1 - timeTaking
    print("Time taking for apriori: " + str(totalTime))
    return assoc_rules_apriori_str,assoc_rules_apriori_str_lift

def exactRule(assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift):
    timeTaking = time.time()
    assoc_rules_str = ""
    min_conf = 1
    PIS1 = []
    PIS = []
    Dict = {}
    f1, sup1 = get_frequent_exact(C[itemset_size], Transactions)

    PISFilePath = input("Enter PIS file path: ")
    f = open(PISFilePath, "w")
    for i in range(0,len(order)):
      f.write(str(random.randrange(0, 5)) + "\n")
    f.close()

    with open(PISFilePath, 'r') as file:
      all_file = file.read().strip()  # Read and remove any extra new line
      all_file_list = all_file.split('\n')  # make a list of lines
      PIS1 = [int(each_int) for each_int in all_file_list]
    
    f = []
    sup = []
    for i in range(0,len(PIS1)):
      if(sup1[i] < PIS1[i]):
        f.append(f1[i])
        sup.append(sup1[i])
        PIS.append(PIS1[i])
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})  

    # List is converting into string  
    def convertList(list1):  
    
        return ' '.join([str(e) for e in list1]) #List comprehension  

    for i in range(len(L[1])):
        # case = {L[1][i] : supp_count_L[1][i]}
        case = {convertList(L[1][i]) : supp_count_L[1][i]}
        Dict.update(case)
    # print(Dict)
    print_PIS_table(L[1], supp_count_L[1], PIS)

    k = itemset_size + 1
    convergence = False
    while not convergence:
      arr = []
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_MIS_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      print_Exact_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      f,sup = get_frequent_exact(C[k], Transactions)
      L.update({k : f})
      
      supp_count_L.update({k : sup})
      if len(L[k]) == 1:
          convergence = True
      else:
          print()
      k += 1

    # for i in range(1, len(L)):
    for i in range(len(arr2)):
      # for j in range(len(arr2[i])):
      s = list(powerset(arr2[i]))
      s.pop()
      for z in s:
        S = set(z)
        X = set(arr2[i])
        X_S = set(X-S)
        sup_x = count_occurences(X , Transactions)
        sup_x_s = count_occurences(X_S, Transactions)
        conf = sup_x / count_occurences(S, Transactions)
        if conf >= min_conf:
          assoc_rules_exact_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
          assoc_rules_apriori_str_exact_lift += write_rules_lift(X, X_S, S, conf, sup_x_s,num_trans)
    
    timeTaking1 = time.time()
    totalTime = timeTaking1 - timeTaking
    print("Time taking for Exact mining: " + str(totalTime))
    return assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift

print("********Rule mining using apriori algorithm********")
assoc_rules_apriori_str,assoc_rules_apriori_str_lift = aprioriAlgo(assoc_rules_apriori_str,assoc_rules_apriori_str_lift)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_apriori_str)
# print("-----------------------Association Rule With Lift-------------------\n")
# print(assoc_rules_apriori_str_lift)
# print("********Rule mining using exact mining********")
# assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift = exactRule(assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift)
# print("-----------------------Association Rule-------------------\n")
# print(assoc_rules_exact_str)
# print("-----------------------Association Rule With Lift-------------------\n")
# print(assoc_rules_apriori_str_exact_lift)

Enter file path: /content/drive/MyDrive/Pamir/tdb.csv
11
********Rule mining using apriori algorithm********
Enter minimum support: 0.6
Itemset | Frequency
['E'] : 4
['K'] : 5
['M'] : 3
['O'] : 3
['Y'] : 3



Table C2:

Itemset | Frequency
['E', 'K'] : 4
['E', 'M'] : 2
['E', 'O'] : 3
['E', 'Y'] : 2
['K', 'M'] : 3
['K', 'O'] : 3
['K', 'Y'] : 3
['M', 'O'] : 1
['M', 'Y'] : 2
['O', 'Y'] : 2



Table L2:

Itemset | Frequency
['E', 'K'] : 4
['E', 'O'] : 3
['K', 'M'] : 3
['K', 'O'] : 3
['K', 'Y'] : 3



Table C3:

Itemset | Frequency
['E', 'K', 'O'] : 3
['K', 'M', 'O'] : 1
['K', 'M', 'Y'] : 2
['K', 'O', 'Y'] : 2



Table L3:

Itemset | Frequency
['E', 'K', 'O'] : 3



Table C4:

Itemset | Frequency



Time taking for apriori: 3.04026460647583
-----------------------Association Rule-------------------

Freq. Itemset: {'E', 'K'}
     Rules: ['E'] -> ['K'] 
     Conf: 1.000 

Freq. Itemset: {'E', 'O'}
     Rules: ['O'] -> ['E'] 
     Conf: 1.000 

Freq. Itemset: {'M', 'K'}
     Rules: ['M'] -> [

In [ ]:
% cd '/content'
!git clone https://github.com/pamirghosh/DataMining

/content
Cloning into 'DataMining'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [ ]:
# Apriori and Exact rule where support >= min_support
import csv
import numpy as np
from python_utils import *
from itertools import combinations, chain
import random

def count_occurences(itemset, Transactions):
  count = 0
  for i in range(len(Transactions)):
    if set(itemset).issubset(set(Transactions[i])):
      count += 1
  return count

def get_frequent_apriori(itemsets, Transactions, min_support, prev_discarded):
  L = []
  supp_count = []
  new_discarded = []
  
  k = len(prev_discarded.keys())

  for s in range(len(itemsets)):
    discarded_before = False
    if k > 0:
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemsets[s])):
          discarded_before = True
          break

    if not discarded_before:
      count = count_occurences(itemsets[s], Transactions)
      if count/len(Transactions) >= min_support:
        L.append(itemsets[s])
        supp_count.append(count)
      else:
        new_discarded.append(itemsets[s])
  return L , supp_count, new_discarded

def get_frequent_exact(itemsets, Transactions):
  L = []
  supp_count = []
  new_discarded = []
  num_trans = len(Transactions)

  for s in range(len(itemsets)):
    count = count_occurences(itemsets[s], Transactions)
    
    L.append(itemsets[s])
    supp_count.append(count)
  return L , supp_count

def load_transactions(path_to_data, order):
    Transactions = []
    with open(path_to_data, 'r') as fid:
      for lines in fid:
        str_line = list(lines.strip().split(','))
        _t = list(np.unique(str_line))
        _t.sort(key = lambda x: order.index(x))
        Transactions.append(_t)
    return Transactions

def print_PIS_table(T, supp_count, PIS):
  print("Itemset|Frequency|PIS")
  for k in range(len(T)):
    print("{} : {} : {}".format(T[k], supp_count[k], PIS[k]))
  print("\n\n")

def print_MIS_table(T, supp_count,supp_count_L, Dict):
  print("Itemset                | Frequency | MIS")
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
  print("\n\n")

arr2 = []

def print_Exact_table(T, supp_count,supp_count_L, Dict):
  print("Exact Itemsets")
  print("-------------------------------------------")
  print("Itemset                | Frequency | MIS")
  count = 0
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    if(supp_count[k] == min(arr)):
      arr2.append(T[k])
      print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
      count += 1
  if(count ==  0):
        print("There is no exact itemsets")
  print("\n\n")

def print_table(T, supp_count):
    print("Itemset | Frequency")
    for k in range(len(T)):
      print("{} : {}".format(T[k], supp_count[k]))
    print("\n\n")

def join_two_itemsets(it1, it2, order):
    it1.sort(key = lambda x: order.index(x))
    it2.sort(key = lambda x: order.index(x))

    for i in range(len(it1) - 1):
      if it1[i] != it2[i]:
        return []
    if order.index(it1[-1]) < order.index(it2[-1]):
      return it1 + [it2[-1]]

    return []

def join_set_itemsets(set_of_its, order):
    C = []
    for i in range(len(set_of_its)):
      for j in range(i+1, len(set_of_its)):
        it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
        if len(it_out) > 0:
          C.append(it_out)
    return C

def powerset(s):
  return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s) + i)))


def write_rules(X, X_S, S, conf, supp, num_trans):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
  out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  
  return out_rules

def write_rules_lift(X, X_S, S, conf, fre,num_trans):
  out_rules = ""
  lift = conf / fre
  if(lift >= 0.25):
    out_rules = ""
    out_rules += "Freq. Itemset: {}\n".format(X)
    out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
    out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
    out_rules += "     Lift: {0:2.3f} \n\n".format(lift)
    # out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  return out_rules

Transactions = []
filePath = input("Enter file path: ")
with open(filePath, newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

for i in range(len(data)):
  Transactions.append(data[i])

num_trans = len(Transactions)
# Transactions
# Transactions
orders = []
orders.append(Transactions[0][0])
for i in range(0,len(Transactions)):
  for j in range(len(Transactions[i])):
    count = 0
    k = 0
    while(k < len(orders)):
      if Transactions[i][j] == orders[k]:
        count += 1
        break
      k += 1
    if count == 0:
      orders.append(Transactions[i][j])

order = sorted(orders)
print(len(orders))
C = {}
L = {}
itemset_size = 1
num_trans = len(Transactions)
supp_count_L = {}
C.update({itemset_size : [ [f] for f in order]})

assoc_rules_apriori_str = ""
assoc_rules_apriori_str_lift = ""
assoc_rules_exact_str = ""
assoc_rules_apriori_str_exact_lift = ""
def aprioriAlgo(assoc_rules_apriori_str,assoc_rules_apriori_str_lift):
    min_support = float(input("Enter minimum support: "))
    min_confidence = 1
    Discarded = { itemset_size : []}
    f, sup, new_discarded = get_frequent_apriori(C[itemset_size], Transactions, min_support, Discarded)
    Discarded.update({itemset_size : new_discarded})
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})
    print_table(L[1], supp_count_L[1])
    k = itemset_size + 1
    convergence = False
    while not convergence:
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_table(C[k], [count_occurences(it, Transactions) for it in C[k]])
      f,sup, new_discarded = get_frequent_apriori(C[k], Transactions, min_support,Discarded)
      Discarded.update({k : new_discarded})
      L.update({k : f})
      supp_count_L.update({k : sup})
      if len(L[k]) == 0:
          convergence = True
      else:
          print("Table L{}:\n".format(k))
          print_table(L[k], supp_count_L[k])
      k += 1
    for i in range(1,len(L)):
      for j in range(len(L[i])):
        s = powerset(L[i][j])
        s.pop()
        for z in s:
          S = set(z)
          X = set(L[i][j])
          X_S = set(X-S)
          sup_x = count_occurences(X , Transactions)
          sup_x_s = count_occurences(X_S, Transactions)
          conf = sup_x / count_occurences(S, Transactions)
          if conf >= min_confidence and sup_x >= min_support:
              assoc_rules_apriori_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
              assoc_rules_apriori_str_lift += write_rules_lift(X, X_S, S, conf, sup_x_s,num_trans)
    return assoc_rules_apriori_str,assoc_rules_apriori_str_lift

def exactRule(assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift):
    assoc_rules_str = ""
    min_conf = 1
    PIS1 = []
    PIS = []
    Dict = {}
    f1, sup1 = get_frequent_exact(C[itemset_size], Transactions)

    PISFilePath = input("Enter PIS file path: ")
    f = open(PISFilePath, "w")
    for i in range(0,len(order)):
      f.write(str(random.randrange(0, 20)) + "\n")
    f.close()

    with open(PISFilePath, 'r') as file:
      all_file = file.read().strip()  # Read and remove any extra new line
      all_file_list = all_file.split('\n')  # make a list of lines
      PIS1 = [int(each_int) for each_int in all_file_list]
    
    f = []
    sup = []
    for i in range(0,len(PIS1)):
      if(sup1[i] < PIS1[i]):
        f.append(f1[i])
        sup.append(sup1[i])
        PIS.append(PIS1[i])
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})  

    # List is converting into string  
    def convertList(list1):  
    
        return ' '.join([str(e) for e in list1]) #List comprehension  

    for i in range(len(L[1])):
        # case = {L[1][i] : supp_count_L[1][i]}
        case = {convertList(L[1][i]) : supp_count_L[1][i]}
        Dict.update(case)
    # print(Dict)
    print_PIS_table(L[1], supp_count_L[1], PIS)

    k = itemset_size + 1
    convergence = False
    while not convergence:
      arr = []
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_MIS_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      print_Exact_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      f,sup = get_frequent_exact(C[k], Transactions)
      L.update({k : f})
      
      supp_count_L.update({k : sup})
      if len(L[k]) == 1:
          convergence = True
      else:
          print()
      k += 1

    # for i in range(1, len(L)):
    for i in range(len(arr2)):
      # for j in range(len(arr2[i])):
      s = list(powerset(arr2[i]))
      s.pop()
      for z in s:
        S = set(z)
        X = set(arr2[i])
        X_S = set(X-S)
        sup_x = count_occurences(X , Transactions)
        sup_x_s = count_occurences(X_S, Transactions)
        conf = sup_x / count_occurences(S, Transactions)
        if conf >= min_conf:
          assoc_rules_exact_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
          assoc_rules_apriori_str_exact_lift += write_rules_lift(X, X_S, S, conf, sup_x_s,num_trans)
    return assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift

print("***Rule mining using apriori algorithm***")
assoc_rules_apriori_str,assoc_rules_apriori_str_lift = aprioriAlgo(assoc_rules_apriori_str,assoc_rules_apriori_str_lift)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_apriori_str)
print("-----------------------Association Rule With Lift-------------------\n")
print(assoc_rules_apriori_str_lift)
# print("***Rule mining using exact mining***")
# assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift = exactRule(assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift)
# print("-----------------------Association Rule-------------------\n")
# print(assoc_rules_exact_str)
# print("-----------------------Association Rule With Lift-------------------\n")
# print(assoc_rules_apriori_str_exact_lift)

In [ ]:
# Apriori and Exact where support <= min_support
import csv
import numpy as np
from python_utils import *
from itertools import combinations, chain
import random

def count_occurences(itemset, Transactions):
  count = 0
  for i in range(len(Transactions)):
    if set(itemset).issubset(set(Transactions[i])):
      count += 1
  return count

def get_frequent_apriori(itemsets, Transactions, min_support, prev_discarded):
  L = []
  supp_count = []
  new_discarded = []
  
  k = len(prev_discarded.keys())

  for s in range(len(itemsets)):
    discarded_before = False
    if k > 0:
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemsets[s])):
          discarded_before = True
          break

    if not discarded_before:
      count = count_occurences(itemsets[s], Transactions)
      if count/len(Transactions) <= min_support:
        L.append(itemsets[s])
        supp_count.append(count)
      else:
        new_discarded.append(itemsets[s])
  return L , supp_count, new_discarded

def get_frequent_exact(itemsets, Transactions):
  L = []
  supp_count = []
  new_discarded = []
  num_trans = len(Transactions)

  for s in range(len(itemsets)):
    count = count_occurences(itemsets[s], Transactions)
    
    L.append(itemsets[s])
    supp_count.append(count)
  return L , supp_count

def load_transactions(path_to_data, order):
    Transactions = []
    with open(path_to_data, 'r') as fid:
      for lines in fid:
        str_line = list(lines.strip().split(','))
        _t = list(np.unique(str_line))
        _t.sort(key = lambda x: order.index(x))
        Transactions.append(_t)
    return Transactions

def print_PIS_table(T, supp_count, PIS):
  print("Itemset|Frequency|PIS")
  for k in range(len(T)):
    print("{} : {} : {}".format(T[k], supp_count[k], PIS[k]))
  print("\n\n")

def print_MIS_table(T, supp_count,supp_count_L, Dict):
  print("Itemset                | Frequency | MIS")
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
  print("\n\n")

arr2 = []

def print_Exact_table(T, supp_count,supp_count_L, Dict):
  print("Exact Itemsets")
  print("-------------------------------------------")
  print("Itemset                | Frequency | MIS")
  count = 0
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    if(supp_count[k] == min(arr)):
      arr2.append(T[k])
      print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
      count += 1
  if(count ==  0):
        print("There is no exact itemsets")
  print("\n\n")

def print_table(T, supp_count):
    print("Itemset | Frequency")
    for k in range(len(T)):
      print("{} : {}".format(T[k], supp_count[k]))
    print("\n\n")

def join_two_itemsets(it1, it2, order):
    it1.sort(key = lambda x: order.index(x))
    it2.sort(key = lambda x: order.index(x))

    for i in range(len(it1) - 1):
      if it1[i] != it2[i]:
        return []
    if order.index(it1[-1]) < order.index(it2[-1]):
      return it1 + [it2[-1]]

    return []

def join_set_itemsets(set_of_its, order):
    C = []
    for i in range(len(set_of_its)):
      for j in range(i+1, len(set_of_its)):
        it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
        if len(it_out) > 0:
          C.append(it_out)
    return C

def powerset(s):
  return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s) + i)))


def write_rules(X, X_S, S, conf, supp, num_trans):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
  out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  
  return out_rules

def write_rules_lift(X, X_S, S, conf, fre,num_trans):
  out_rules = ""
  lift = conf / fre
  if(lift >= 0.25):
    out_rules = ""
    out_rules += "Freq. Itemset: {}\n".format(X)
    out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
    out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
    out_rules += "     Lift: {0:2.3f} \n\n".format(lift)
    # out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  return out_rules

Transactions = []
filePath = input("Enter file path: ")
with open(filePath, newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

for i in range(len(data)):
  Transactions.append(data[i])

num_trans = len(Transactions)
# Transactions
# Transactions
orders = []
orders.append(Transactions[0][0])
for i in range(0,len(Transactions)):
  for j in range(len(Transactions[i])):
    count = 0
    k = 0
    while(k < len(orders)):
      if Transactions[i][j] == orders[k]:
        count += 1
        break
      k += 1
    if count == 0:
      orders.append(Transactions[i][j])

order = sorted(orders)
print(len(orders))
C = {}
L = {}
itemset_size = 1
num_trans = len(Transactions)
supp_count_L = {}
C.update({itemset_size : [ [f] for f in order]})

assoc_rules_apriori_str = ""
assoc_rules_apriori_str_lift = ""
assoc_rules_exact_str = ""
assoc_rules_apriori_str_exact_lift = ""
def aprioriAlgo(assoc_rules_apriori_str,assoc_rules_apriori_str_lift):
    min_support = float(input("Enter minimum support: "))
    min_confidence = 1
    Discarded = { itemset_size : []}
    f, sup, new_discarded = get_frequent_apriori(C[itemset_size], Transactions, min_support, Discarded)
    Discarded.update({itemset_size : new_discarded})
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})
    print_table(L[1], supp_count_L[1])
    k = itemset_size + 1
    convergence = False
    while not convergence:
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_table(C[k], [count_occurences(it, Transactions) for it in C[k]])
      f,sup, new_discarded = get_frequent_apriori(C[k], Transactions, min_support,Discarded)
      Discarded.update({k : new_discarded})
      L.update({k : f})
      supp_count_L.update({k : sup})
      if len(L[k]) == 0:
          convergence = True
      else:
          print("Table L{}:\n".format(k))
          print_table(L[k], supp_count_L[k])
      k += 1
    for i in range(1,len(L)):
      for j in range(len(L[i])):
        s = powerset(L[i][j])
        s.pop()
        for z in s:
          S = set(z)
          X = set(L[i][j])
          X_S = set(X-S)
          sup_x = count_occurences(X , Transactions)
          sup_x_s = count_occurences(X_S, Transactions)
          co = count_occurences(S, Transactions)
          if co > 0:
            conf = sup_x / co
            if conf >= min_confidence:
              assoc_rules_apriori_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
              assoc_rules_apriori_str_lift += write_rules_lift(X, X_S, S, conf, sup_x_s,num_trans)
    return assoc_rules_apriori_str,assoc_rules_apriori_str_lift

def exactRule(assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift):
    assoc_rules_str = ""
    min_conf = 1
    PIS1 = []
    PIS = []
    Dict = {}
    f1, sup1 = get_frequent_exact(C[itemset_size], Transactions)

    PISFilePath = input("Enter PIS file path: ")
    f = open(PISFilePath, "w")
    for i in range(0,len(order)):
      f.write(str(random.randrange(0, 20)) + "\n")
    f.close()

    with open(PISFilePath, 'r') as file:
      all_file = file.read().strip()  # Read and remove any extra new line
      all_file_list = all_file.split('\n')  # make a list of lines
      PIS1 = [int(each_int) for each_int in all_file_list]
    
    f = []
    sup = []
    for i in range(0,len(PIS1)):
      if(sup1[i] < PIS1[i]):
        f.append(f1[i])
        sup.append(sup1[i])
        PIS.append(PIS1[i])
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})  

    # List is converting into string  
    def convertList(list1):  
    
        return ' '.join([str(e) for e in list1]) #List comprehension  

    for i in range(len(L[1])):
        # case = {L[1][i] : supp_count_L[1][i]}
        case = {convertList(L[1][i]) : supp_count_L[1][i]}
        Dict.update(case)
    # print(Dict)
    print_PIS_table(L[1], supp_count_L[1], PIS)

    k = itemset_size + 1
    convergence = False
    while not convergence:
      arr = []
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_MIS_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      print_Exact_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      f,sup = get_frequent_exact(C[k], Transactions)
      L.update({k : f})
      
      supp_count_L.update({k : sup})
      if len(L[k]) == 1:
          convergence = True
      else:
          print()
      k += 1

    # for i in range(1, len(L)):
    for i in range(len(arr2)):
      # for j in range(len(arr2[i])):
      s = list(powerset(arr2[i]))
      s.pop()
      for z in s:
        S = set(z)
        X = set(arr2[i])
        X_S = set(X-S)
        sup_x = count_occurences(X , Transactions)
        sup_x_s = count_occurences(X_S, Transactions)
        conf = sup_x / count_occurences(S, Transactions)
        if conf >= min_conf:
          assoc_rules_exact_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
          assoc_rules_apriori_str_exact_lift += write_rules_lift(X, X_S, S, conf, sup_x_s,num_trans)
    return assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift

# print("***Rule mining using apriori algorithm***")
# assoc_rules_apriori_str,assoc_rules_apriori_str_lift = aprioriAlgo(assoc_rules_apriori_str,assoc_rules_apriori_str_lift)
# print("-----------------------Association Rule-------------------\n")
# print(assoc_rules_apriori_str)
# print("-----------------------Association Rule With Lift-------------------\n")
# print(assoc_rules_apriori_str_lift)
print("***Rule mining using exact mining***")
assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift = exactRule(assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_exact_str)
print("-----------------------Association Rule With Lift-------------------\n")
print(assoc_rules_apriori_str_exact_lift)

In [ ]:
# Apriori and Exact where support <= min_support
import csv
import numpy as np
from python_utils import *
from itertools import combinations, chain
import random

def count_occurences(itemset, Transactions):
  count = 0
  for i in range(len(Transactions)):
    if set(itemset).issubset(set(Transactions[i])):
      count += 1
  return count

def get_frequent_apriori(itemsets, Transactions, min_support, prev_discarded):
  L = []
  supp_count = []
  new_discarded = []
  
  k = len(prev_discarded.keys())

  for s in range(len(itemsets)):
    discarded_before = False
    if k > 0:
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemsets[s])):
          discarded_before = True
          break

    if not discarded_before:
      count = count_occurences(itemsets[s], Transactions)
      if count/len(Transactions) <= min_support:
        L.append(itemsets[s])
        supp_count.append(count)
      else:
        new_discarded.append(itemsets[s])
  return L , supp_count, new_discarded

def get_frequent_exact(itemsets, Transactions):
  L = []
  supp_count = []
  new_discarded = []
  num_trans = len(Transactions)

  for s in range(len(itemsets)):
    count = count_occurences(itemsets[s], Transactions)
    
    L.append(itemsets[s])
    supp_count.append(count)
  return L , supp_count

def load_transactions(path_to_data, order):
    Transactions = []
    with open(path_to_data, 'r') as fid:
      for lines in fid:
        str_line = list(lines.strip().split(','))
        _t = list(np.unique(str_line))
        _t.sort(key = lambda x: order.index(x))
        Transactions.append(_t)
    return Transactions

def print_PIS_table(T, supp_count, PIS):
  print("Itemset|Frequency|PIS")
  for k in range(len(T)):
    print("{} : {} : {}".format(T[k], supp_count[k], PIS[k]))
  print("\n\n")

def print_MIS_table(T, supp_count,supp_count_L, Dict):
  print("Itemset                | Frequency | MIS")
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
  print("\n\n")

arr2 = []

def print_Exact_table(T, supp_count,supp_count_L, Dict):
  print("Exact Itemsets")
  print("-------------------------------------------")
  print("Itemset                | Frequency | MIS")
  count = 0
  for k in range(len(T)):
    arr = []
    for i in range(len(T[k])):
      for key, value in Dict.items():
        if(T[k][i] == key):
          arr.append(value)
    if(supp_count[k] == min(arr)):
      arr2.append(T[k])
      print("{} :       {}    :   {}".format(T[k], supp_count[k], min(arr)))
      count += 1
  if(count ==  0):
        print("There is no exact itemsets")
  print("\n\n")

def print_table(T, supp_count):
    print("Itemset | Frequency")
    for k in range(len(T)):
      print("{} : {}".format(T[k], supp_count[k]))
    print("\n\n")

def join_two_itemsets(it1, it2, order):
    it1.sort(key = lambda x: order.index(x))
    it2.sort(key = lambda x: order.index(x))

    for i in range(len(it1) - 1):
      if it1[i] != it2[i]:
        return []
    if order.index(it1[-1]) < order.index(it2[-1]):
      return it1 + [it2[-1]]

    return []

def join_set_itemsets(set_of_its, order):
    C = []
    for i in range(len(set_of_its)):
      for j in range(i+1, len(set_of_its)):
        it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
        if len(it_out) > 0:
          C.append(it_out)
    return C

def powerset(s):
  return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s) + i)))


def write_rules(X, X_S, S, conf, supp, num_trans):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
  out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  
  return out_rules

def write_rules_lift(X, X_S, S, conf, fre,num_trans):
  out_rules = ""
  lift = conf / fre
  if(lift >= 0.25):
    out_rules = ""
    out_rules += "Freq. Itemset: {}\n".format(X)
    out_rules += "     Rules: {} -> {} \n".format(list(S), list(X_S))
    out_rules += "     Conf: {0:2.3f} \n\n".format(conf)
    out_rules += "     Lift: {0:2.3f} \n\n".format(lift)
    # out_rules += "     Supp: {0:2.3f} \n\n".format(supp / num_trans)
  return out_rules

Transactions = []
filePath = input("Enter file path: ")
with open(filePath, newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

for i in range(len(data)):
  Transactions.append(data[i])

num_trans = len(Transactions)
# Transactions
# Transactions
orders = []
orders.append(Transactions[0][0])
for i in range(0,len(Transactions)):
  for j in range(len(Transactions[i])):
    count = 0
    k = 0
    while(k < len(orders)):
      if Transactions[i][j] == orders[k]:
        count += 1
        break
      k += 1
    if count == 0:
      orders.append(Transactions[i][j])

order = sorted(orders)
C = {}
L = {}
itemset_size = 1
num_trans = len(Transactions)
supp_count_L = {}
C.update({itemset_size : [ [f] for f in order]})

assoc_rules_apriori_str = ""
assoc_rules_apriori_str_lift = ""
assoc_rules_exact_str = ""
assoc_rules_apriori_str_exact_lift = ""
min_support = float(input("Enter minimum support: "))
def aprioriAlgo(assoc_rules_apriori_str,assoc_rules_apriori_str_lift):
    min_confidence = 1
    Discarded = { itemset_size : []}
    f, sup, new_discarded = get_frequent_apriori(C[itemset_size], Transactions, min_support, Discarded)
    Discarded.update({itemset_size : new_discarded})
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})
    print_table(L[1], supp_count_L[1])
    k = itemset_size + 1
    convergence = False
    while not convergence:
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_table(C[k], [count_occurences(it, Transactions) for it in C[k]])
      f,sup, new_discarded = get_frequent_apriori(C[k], Transactions, min_support,Discarded)
      Discarded.update({k : new_discarded})
      L.update({k : f})
      supp_count_L.update({k : sup})
      if len(L[k]) == 0:
          convergence = True
      else:
          print("Table L{}:\n".format(k))
          print_table(L[k], supp_count_L[k])
      k += 1
    for i in range(1,len(L)):
      for j in range(len(L[i])):
        s = powerset(L[i][j])
        s.pop()
        for z in s:
          S = set(z)
          X = set(L[i][j])
          X_S = set(X-S)
          sup_x = count_occurences(X , Transactions)
          sup_x_s = count_occurences(X_S, Transactions)
          co = count_occurences(S, Transactions)
          if co > 0:
            conf = sup_x / co
            if conf >= min_confidence:
              assoc_rules_apriori_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
              assoc_rules_apriori_str_lift += write_rules_lift(X, X_S, S, conf, sup_x_s,num_trans)
    return assoc_rules_apriori_str,assoc_rules_apriori_str_lift

def exactRule(assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift):
    assoc_rules_str = ""
    min_conf = 1
    PIS1 = []
    PIS = []
    Dict = {}
    f1, sup1 = get_frequent_exact(C[itemset_size], Transactions)
    max_PIS = min_support * num_trans

    PISFilePath = input("Enter PIS file path: ")
    f = open(PISFilePath, "w")
    for i in range(0,len(order)):
      f.write(str(random.randrange(0, max_PIS)) + "\n")
    f.close()

    with open(PISFilePath, 'r') as file:
      all_file = file.read().strip()  # Read and remove any extra new line
      all_file_list = all_file.split('\n')  # make a list of lines
      PIS1 = [int(each_int) for each_int in all_file_list]
    
    f = []
    sup = []
    for i in range(0,len(PIS1)):
      if(sup1[i] < PIS1[i]):
        f.append(f1[i])
        sup.append(sup1[i])
        PIS.append(PIS1[i])
    L.update({itemset_size : f})
    supp_count_L.update({itemset_size : sup})  

    # List is converting into string  
    def convertList(list1):  
    
        return ' '.join([str(e) for e in list1]) #List comprehension  

    for i in range(len(L[1])):
        # case = {L[1][i] : supp_count_L[1][i]}
        case = {convertList(L[1][i]) : supp_count_L[1][i]}
        Dict.update(case)
    # print(Dict)
    print_PIS_table(L[1], supp_count_L[1], PIS)

    k = itemset_size + 1
    convergence = False
    while not convergence:
      arr = []
      C.update({ k: join_set_itemsets(L[k-1], order)})
      print("Table C{}:\n".format(k))
      print_MIS_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      print_Exact_table(C[k], [count_occurences(it, Transactions) for it in C[k]], supp_count_L[1], Dict)
      f,sup = get_frequent_exact(C[k], Transactions)
      L.update({k : f})
      supp_count_L.update({k : sup})
      if len(L[k]) == 1 or len(L[k]) == 0:
          convergence = True
      else:
          print()
      k += 1

    # for i in range(1, len(L)):
    for i in range(len(arr2)):
      # for j in range(len(arr2[i])):
      s = list(powerset(arr2[i]))
      s.pop()
      for z in s:
        S = set(z)
        X = set(arr2[i])
        X_S = set(X-S)
        sup_x = count_occurences(X , Transactions)
        sup_x_s = count_occurences(X_S, Transactions)
        conf = sup_x / count_occurences(S, Transactions)
        if conf >= min_conf:
          assoc_rules_exact_str += write_rules(X, X_S, S, conf, sup_x,num_trans)
          assoc_rules_apriori_str_exact_lift += write_rules_lift(X, X_S, S, conf, sup_x_s,num_trans)
    return assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift

print("***Rule mining using apriori algorithm***")
assoc_rules_apriori_str,assoc_rules_apriori_str_lift = aprioriAlgo(assoc_rules_apriori_str,assoc_rules_apriori_str_lift)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_apriori_str)
print("-----------------------Association Rule With Lift-------------------\n")
print(assoc_rules_apriori_str_lift)
print("***Rule mining using exact mining***")
assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift = exactRule(assoc_rules_exact_str,assoc_rules_apriori_str_exact_lift)
print("-----------------------Association Rule-------------------\n")
print(assoc_rules_exact_str)
print("-----------------------Association Rule With Lift-------------------\n")
print(assoc_rules_apriori_str_exact_lift)

In [ ]:
# Putnam's work for software development
import matplotlib.pyplot as plt

effort = int(input("Enter effort(developers) need to develop the software: "))
time = int(input("Enter estimated time for develop a software(in months): "))
# time = 0.3935 * effort
# print("Estimated time for develop a software(in months): " + str(time))
tc = [2,8,11]

def relay_graph():
  print("\n")
  
  plt.legend(loc="upper right")
  plt.xlabel("time") 
  plt.ylabel("product size")
  plt.title("r_curve for putnam")
  plt.show()

def putnam(tc,effort,time): 
  return tc * pow(effort, 1/3) * pow(time, 4/3)

colors = {
      2: 'red',
      8: 'orange',
      11: 'green'
  }

# for records in tc:
#   points = []
#   kloc = str(putnam(records,effort,time))
#   project_duration = 3 * int(time)
#   for timeTaken in range(0, project_duration):
#     points.append(round(putnam(records, effort, timeTaken), 5))
#   plt.plot(points, color=colors[records], linewidth=4, label=f"TC: {records} | KLOC: {kloc}")
#   relay_graph()

points = []
k = []
for records in tc:
  a = []
  kloc = str(putnam(records,effort,time))
  project_duration = 3 * int(time)
  for timeTaken in range(0, project_duration):
    a.append(round(putnam(records, effort, timeTaken), 5))
  
  points.append(a)
  k.append(kloc)
  plt.plot(a, color=colors[records], linewidth=4, label=f"TC: {records} | KLOC: {kloc}")
  relay_graph()
i = 0
for records in tc:
  plt.plot(points[i], color=colors[records], linewidth=4, label=f"TC: {records} | KLOC: {k[i]}")
  i += 1
relay_graph()


In [ ]:
# Norden's Work for research and development
import matplotlib.pyplot as plt

t = int(input("Enter effort time(t) in months: "))
K = int(input("Enter area under the curve: "))
Td = int(input("Enter the time at which the curve attains its maximum values(td) in months: ")) 
def norden(K,Td,t):
  e = 2.7182
  c1 = pow(Td,2)
  c2 = pow(t,2)
  c3 = -c2 / (2 * c1)
  c4 = pow(e,c3)
  return (K / c1) * t * c4
def relayGraph(K,Td,t):
  project_duration = 3 * int(Td)
  y = []
  for time in range(0, project_duration):
    y.append(round(norden(K, Td, time), 5) )
  plt.plot(y, color="blue", linewidth=4, label="Effort respect to time")
  plt.vlines(
        x=Td,
        ymin=0,
        ymax=round(norden(K, Td, Td), 5),
        color="blue",
        linestyle="dashed",
        linewidth=4,
        label=f"td ({Td})",
    )
  plt.vlines(
        x=t,
        ymin=0,
        ymax=round(norden(K, Td, t), 5),
        color="yellow",
        linestyle="dashed",
        linewidth=4,
        label="t"
    )
  plt.legend(loc="upper right")
  plt.xlabel("Time")
  plt.ylabel("Effort")
  plt.title("r_curve for norden")
  plt.show()
print(f"Effort(no of developers) required at time {t} months : {round(norden(K, Td, t), 5)}")
relayGraph(K,Td,t)

In [ ]:
# Jensen's work for software development
import matplotlib.pyplot as plt

effort = int(input("Enter effort(developers) need to develop the software: "))
time = int(input("Enter estimated time for develop a software(in months): "))
# time = 0.3935 * effort
# print("Estimated time for develop a software(in months): " + str(time))
tc = [2,8,11]

def relay_graph():
  print("\n")
  
  plt.legend(loc="upper right")
  plt.xlabel("time") 
  plt.ylabel("product size")
  plt.title("r_curve for jensen")
  plt.show()

def jensen(tc,effort,time): 
  return tc * time * pow(effort, 1/2)

colors = {
      2: 'red',
      8: 'orange',
      11: 'green'
  }

# for records in tc:
#   points = []
#   kloc = str(putnam(records,effort,time))
#   project_duration = 3 * int(time)
#   for timeTaken in range(0, project_duration):
#     points.append(round(putnam(records, effort, timeTaken), 5))
#   plt.plot(points, color=colors[records], linewidth=4, label=f"TC: {records} | KLOC: {kloc}")
#   relay_graph()

points = []
k = []
for records in tc:
  a = []
  kloc = str(jensen(records,effort,time))
  project_duration = 3 * int(time)
  for timeTaken in range(0, project_duration):
    a.append(round(jensen(records, effort, timeTaken), 5))
  
  points.append(a)
  k.append(kloc)
  plt.plot(a, color=colors[records], linewidth=4, label=f"TC: {records} | KLOC: {kloc}")
  relay_graph()
i = 0
for records in tc:
  plt.plot(points[i], color=colors[records], linewidth=4, label=f"TC: {records} | KLOC: {k[i]}")
  i += 1
relay_graph()

In [ ]:
# Basic COCOMO Model
import matplotlib.pyplot as plt
kloc = int(input("Enter total number of lines of code(KLOC): "))

colors = {
      0: 'red',
      1: 'orange',
      2: 'green'
  }
effort = 0
def relay_graph_effort(a,b,c,d,kloc,color):
  effort,time = cocomo(a,b,c,d,kloc,0)
  effortArr = []
  klocArr = []
  klocnew = kloc + 10
  for i in range(1,kloc + 1):
    effortArr.append(cocomo(a,b,c,d,i,1))
    klocArr.append(i)
  
  plt.plot(klocArr,effortArr, color=colors[color], linewidth=4, label=f"E: {effort} | T: {time}")
  draw_curve(1)
  return effortArr
  # print(effortArr)
  # print(klocArr)

def relay_graph_time(a,b,c,d,kloc,color):
  effort,time = cocomo(a,b,c,d,kloc,3)
  timeArr = []
  klocArr = []
  klocnew = kloc + 10
  for i in range(1,kloc + 1):
    timeArr.append(cocomo(a,b,c,d,i,2))
    klocArr.append(i)
  
  plt.plot(klocArr,timeArr, color=colors[color], linewidth=4, label=f"E: {effort} | T: {time}")
  draw_curve(0)
  return timeArr

def draw_curve(flag):
  plt.legend(loc="upper right")
  plt.xlabel("KLOC") 
  if flag == 1:
    plt.ylabel("Estimated Effort")
    plt.title("Effort versus product size")
  else:
    plt.ylabel("Estimated Time")
    plt.title("Time versus product size")
  plt.show()
  print("\n")

def cocomo(a,b,c,d,kloc,flag):
  
  effort = round((a * pow(kloc,b)), 4)
  time = round((c * pow(effort,d)), 4)
  personRequired = round((effort / time), 4)
  productivity = round((kloc / effort),4)
  if flag == 1:
    return effort
  if flag == 2:
    return time
  if flag == 3:
    return effort,time
  print("Effort: " + str(effort)+ " PM")
  print("Time: " + str(time) + " months")
  print("Average person required: " + str(personRequired) + " persons")
  print("Productivity: " + str(productivity) + " KLOC/PM")
  return effort,time

effortArr = []
klocArr = []
timeArr = []
for i in range(0,3):
  if(i == 0):
    print("In Organic Mode")
    effortArr.append(relay_graph_effort(2.4,1.05,2.5,0.38,kloc, 0))
    timeArr.append(relay_graph_time(2.4,1.05,2.5,0.38,kloc, 0))
  elif(i == 1):
    print("In Semi Detached Mode")
    effortArr.append(relay_graph_effort(3.0	,1.12	,2.5,	0.35, kloc, 1))
    timeArr.append(relay_graph_time(3.0	,1.12	,2.5,	0.35, kloc, 1))
  elif(i== 2):
    print("In Embedded Mode")
    effortArr.append(relay_graph_effort(3.6	,1.20,	2.5,	0.32, kloc, 2))
    timeArr.append(relay_graph_time(3.6	,1.20,	2.5,	0.32, kloc, 2))
for i in range(1,kloc + 1):
    klocArr.append(i)

for i in range(0,3):
  plt.plot(klocArr,effortArr[i], color=colors[i], linewidth=4)
  i += 1
draw_curve(1)

for i in range(0,3):
  plt.plot(klocArr,timeArr[i], color=colors[i], linewidth=4)
  i += 1
draw_curve(0)


# Intermediate Cocomo Model 


In [ ]:
# Intermediate Cocomo Model 
import matplotlib.pyplot as plt

kloc = int(input("Enter total number of lines of code: \n"))
constants = []

colors = {
      0: 'red',
      1: 'orange',
      2: 'green'
  }
effort = 0

required_software_reliability = [0.75,0.88,1.00,1.15,1.40]
size_of_application_database	=	[0,0.94,1.00,1.08,1.16]
complexity_of_the_product =	[0.70,0.85,1.00,1.15,1.30]
runtime_performance_constraints = [0,0,1.00,	1.11,	1.30]
memory_constraints = [0,0,1.00,1.06,1.21]
volatility_of_the_virtual_machine_environment	=	[0,0.87,1.00,1.15,1.30]
required_turnabout_time	=	[0,0.94,1.00,1.07,1.15]
analyst_capability	= [1.46,1.19,1.00,0.86,0.71]
applications_experience	= [1.29,1.13,1.00,0.91,0.82]
software_engineer_capability	= [1.42,	1.17,	1.00,	0.86,	0.70]
virtual_machine_experience	= [1.21,	1.10,	1.00,	0.90, 0]
programming_language_experience	= [1.14,	1.07,	1.00,	0.95, 0]		
application_of_software_engineering_methods =	[1.24,	1.10,	1.00,	0.91,	0.82]
use_of_s# Intermediate Cocomo Model oftware_tools =	[1.24,	1.10,	1.00,	0.91, 0.83]
required_development_schedule	= [1.23,	1.08,	1.00,	1.04,	1.10]

informations = ["Select Required Software Reliability: ",
                "Select Size of Application Database(Select between Low to Very High): ",
                "Select Complexity of The Product: ",
                "Select Runtime Performance Constraints(Select between Nominal to Very High): ",
                "Select Memory Constraints(Select between Nominal to Very High): ",
                "Select Volatility of the virtual machine environment(Select between Low to Very High): ",
                "Select Required turnabout time(Select between Low to Very High): ",
                "Select Analyst capability: ",
                "Select Applications experience: ",
                "Select Software engineer capability: ",
                "Select Virtual machine experience(Select between Very Low to High): ",
        # Intermediate Cocomo Model         "Select Programming language experience(Select between Very Low to High): ",
                "Select Application of software engineering methods: ",
                "Select Use of software tools: ",
                "Select Required development schedule: "
                ]

def selectConstants(option,value):
  if option == 0:
      constants.append(required_software_reliability[value])
  elif option == 1:
      constants.append(size_of_application_database[value])
  elif option == 2:
      constants.append(complexity_of_the_product[value])
  elif o# Intermediate Cocomo Model ption == 3:
      constants.append(runtime_performance_constraints[value])
  elif option == 4:
      constants.append(memory_constraints[value])
  elif option == 5:
      constants.append(volatility_of_the_virtual_machine_environment[value])
  elif option == 6:
      constants.append(required_turnabout_time[value])
  elif option == 7:
      constants.append(analyst_capability[value])
  elif option == 8:
      constants.append(applications_experience[value])
  elif option == 9:
      constants.append(software_engineer_capability[value])
  elif o# Intermediate Cocomo Model ption == 10:
      constants.append(virtual_machine_experience[value])
  elif option == 11:
      constants.append(programming_language_experience[value])
  elif option == 12:
      constants.append(application_of_software_engineering_methods[value])
  elif option == 13:
      constants.append(use_of_software_tools[value])
  elif option == 14:
      constants.append(required_development_schedule[value])
  
def relay_graph_effort(a,b,kloc,color,EAF):
  effort = cocomo(a,b,kloc,0,EAF)
  effortArr = []
  klocArr = []
  klocnew = kloc + 10
  for i in range(1,kloc + 1):
    effortArr.append(cocomo(a,b,i,1,EAF))
    klocArr.append(i)
  
  plt.plot(klocArr,effortArr, color=colors[color], linewidth=4, label=f"E: {effort}")
  draw_curve(1)
  return effortArr
  # print(effortArr)
  # print(klocArr)

def draw_curve(flag):
  plt.legend(loc="upper right")
  plt.xlabel("KLOC") 
  if flag == 1:
    plt.ylabel("Estimated Effort")
    plt.title("Effort versus product size")
  else:
    plt.ylabel("Estimated Time")
    plt.title("Time versus product size")
  plt.show()
  print("\n")

def cocomo(a,b,kloc,flag,EAF):
  effort = round((a * pow(kloc,b)), 4) * EAF
  # time = round((c * pow(effort,d)), 4)
  # personRequired = round((effort / time), 4)
  # productivity = round((kloc / effort),4)
  if flag == 1:
    return effort
  # if f# Intermediate Cocomo Model lag == 2:
  #   return time
  # if flag == 2:
    # return effort,time
  print("Effort: " + str(effort)+ " PM")
  # print("Time: " + str(time) + " months")
  # print("Average person required: " + str(personRequired) + " persons")
  # print("Productivity: " + str(productivity) + " KLOC/PM")
  # return effort,
  return effort

EAF  = 1
for i in range(0,15):
  
  print("\n")
  print("Very Low- Press 0\nLow - Press 1\nNominal - Press 2\nHigh - Press 3\nVery High - Press 4")
  flag = int(input(informations[i]))
  select# Intermediate Cocomo Model Constants(i,flag)

print("----------------------------------------------------------------------\n")
for i in constants:
      EAF = EAF * i

print("Effort Adjustment Factor: " + str(EAF) + "\n")
effortArr = []
klocArr = []
timeArr = []
for i in range(0,3):
  if(i == 0):
    print("In Organic Mode")
    effortArr.append(relay_graph_effort(3.2,1.05,kloc, 0,EAF))
    # timeArr.append(relay_graph_time(2.4,1.05,2.5,0.38,kloc, 0))
  elif(i == 1):
    print("In Semi Detached Mode")
    effortArr.append(relay_graph_effort(3.0	,1.12, kloc, 1,EAF))
    # timeArr.append(relay_graph_time(3.0	,1.12	,2.5,	0.35, kloc, 1))
  elif(i== 2):
    print("In Embedded Mode")
    effortArr.append(relay_graph_effort(2.8	,1.20, kloc, 2,EAF))
    # timeArr.append(relay_graph_time(3.6	,1.20,	2.5,	0.32, kloc, 2))
for i in range(1,kloc + 1):
    klocArr.append(i)

for i in range(0,3):
  plt.plot(klocArr,effortArr[i], color=colors[i], linewidth=4)
  i += 1
draw_curv
  print("\n")
  prie(1)

# for i in range(0,3):
#   plt.plot(klocArr,timeArr[i], color=colors[i], linewidth=4)
#   i += 1
# draw_curve(0)

# Detailed Cocomo Model 

In [1]:
# Detailed Cocomo Model 
import matplotlib.pyplot as plt
import numpy

kloc = int(input("Enter total number of lines of code: \n"))
constants = []

colors = {
      0: 'red',
      1: 'orange',
      2: 'green'
  }
effort = 0

required_software_reliability = [0.75,0.88,1.00,1.15,1.40]
size_of_application_database	=	[0,0.94,1.00,1.08,1.16]
complexity_of_the_product =	[0.70,0.85,1.00,1.15,1.30]
runtime_performance_constraints = [0,0,1.00,	1.11,	1.30]
memory_constraints = [0,0,1.00,1.06,1.21]
volatility_of_the_virtual_machine_environment	=	[0,0.87,1.00,1.15,1.30]
required_turnabout_time	=	[0,0.94,1.00,1.07,1.15]
analyst_capability	= [1.46,1.19,1.00,0.86,0.71]
applications_experience	= [1.29,1.13,1.00,0.91,0.82]
software_engineer_capability	= [1.42,	1.17,	1.00,	0.86,	0.70]
virtual_machine_experience	= [1.21,	1.10,	1.00,	0.90, 0]
programming_language_experience	= [1.14,	1.07,	1.00,	0.95, 0]		
application_of_software_engineering_methods =	[1.24,	1.10,	1.00,	0.91,	0.82]
use_of_software_tools =	[1.24,	1.10,	1.00,	0.91, 0.83]
required_development_schedule	= [1.23,	1.08,	1.00,	1.04,	1.10]

organic_small_up = [0.06,0.16,0.26,0.42,0.96]
organic_mid_up = [0.06,0.16,0.24,0.38,0.22]
s_det_mid_up = [0.07,0.17,0.25,0.33,0.25]
s_det_large_up = [0.07,0.17,0.24,0.31,0.28]
embaded_large_up = [0.08,0.18,0.25,0.26,0.31]
embaded_xlarge_up = [0.08,0.18,0.24,0.24,0.34]

organic_small_tp = [0.10,0.19,0.24,0.39,0.18]
organic_mid_tp = [0.12,0.19,0.21,0.34,0.26]
s_det_mid_tp = [0.20,0.26,0.21,0.27,0.26]
s_det_large_tp = [0.22,0.27,0.19,0.25,0.29]
embaded_large_tp = [0.36,0.36,0.18,0.18,0.28]
embaded_xlarge_tp = [0.40,0.38,0.16,0.16,0.30]

sofConst = ["Plan & Requirement: ",
            "Design: ",
            "Detail Design: ",
            "Code & Test: ",
            "Integration & Test: "
            ]

informations = ["Select Required Software Reliability: ",
                "Select Size of Application Database(Select between Low to Very High): ",
                "Select Complexity of The Product: ",
                "Select Runtime Performance Constraints(Select between Nominal to Very High): ",
                "Select Memory Constraints(Select between Nominal to Very High): ",
                "Select Volatility of the virtual machine environment(Select between Low to Very High): ",
                "Select Required turnabout time(Select between Low to Very High): ",
                "Select Analyst capability: ",
                "Select Applications experience: ",
                "Select Software engineer capability: ",
                "Select Virtual machine experience(Select between Very Low to High): ",
                "Select Programming language experience(Select between Very Low to High): ",
                "Select Application of software engineering methods: ",
                "Select Use of software tools: ",
                "Select Required development schedule: "
                ]

def selectConstants(option,value):
  if option == 0:
      constants.append(required_software_reliability[value])
  elif option == 1:
      constants.append(size_of_application_database[value])
  elif option == 2:
      constants.append(complexity_of_the_product[value])
  elif option == 3:
      constants.append(runtime_performance_constraints[value])
  elif option == 4:
      constants.append(memory_constraints[value])
  elif option == 5:
      constants.append(volatility_of_the_virtual_machine_environment[value])
  elif option == 6:
      constants.append(required_turnabout_time[value])
  elif option == 7:
      constants.append(analyst_capability[value])
  elif option == 8:
      constants.append(applications_experience[value])
  elif option == 9:
      constants.append(software_engineer_capability[value])
  elif option == 10:
      constants.append(virtual_machine_experience[value])
  elif option == 11:
      constants.append(programming_language_experience[value])
  elif option == 12:
      constants.append(application_of_software_engineering_methods[value])
  elif option == 13:
      constants.append(use_of_software_tools[value])
  elif option == 14:
      constants.append(required_development_schedule[value])
  

def detailedCocomo(E,T):
  cons = []
  # cons2 = []
  cons3 = []
  # cons4 = []
  if(E < 32000):
    for j in organic_small_up:
      flag = round((j * E),4)
      cons.append(flag)
    for k in organic_small_tp:
      flag = round((j * T),4)
      cons3.append(flag)
    return cons,cons3
  elif(E >= 32000 and E < 128000):
    for j in organic_mid_up:
      flag = round((j * E),4)
      cons.append(flag)
    for k in organic_mid_tp:
      flag = round((j * T),4)
      cons3.append(flag)
    return cons,cons3
  elif(E >= 128000 and E < 320000):
    for j in s_det_large_up:
      flag = round((j * E),4)
      cons.append(flag)
    for k in s_det_large_tp:
        flag = round((j * T),4)
        cons3.append(flag)
    return cons,cons3
  elif(E >= 320000):
    for j in embaded_xlarge_up:
      flag = round((j * E),4)
      cons.append(flag)
    for k in embaded_xlarge_tp:
      flag = round((j * T),4)
      cons3.append(flag)
    return cons,cons3

def relay_graph(a,b,c,d,kloc,color,EAF):
  cons = []
  cons2 = []
  effort,time = cocomo(a,b,c,d,kloc,0,EAF)
  print(f"Effort required {effort} PM")
  print(f"Time required {time} months")
  cons,cons2 = detailedCocomo(effort,time)

  print("\n")
  print("Effort required in different phases")
  print("************************************************************\n")
  for i in range(0,5):
    print(sofConst[i] + str(cons[i]) + " PM")
    
  print("\n")
  print("Development time in different phases")
  print("************************************************************\n")
  for i in range(0,5):
    print(sofConst[i] + str(cons2[i]) + " Months")
  # effortArr = []
  # effortPoints = []
  # klocArr = []
  # klocnew = kloc + 10
  # l2 = []
  # for i in range(1,kloc + 1):
  #   pointEffort = cocomo(a,b,c,d,i,1,EAF)
  #   effortPoints.append(detailedCocomo(pointEffort,time))
  #   klocArr.append(i)
  
  # l2 = transpose(effortPoints, l2)
  # for i in range(0,5):
  #   plt.plot(klocArr,l2[i], color=colors[color], linewidth=4, label=f"E: {effort}")
  # draw_curve(1)
  # return effortArr

def transpose(l1, l2):
 
    l2 = list(map(list, zip(*l1)))
    return l2


def draw_curve(flag):
  plt.legend(loc="upper right")
  plt.xlabel("KLOC") 
  if flag == 1:
    plt.ylabel("Estimated Effort")
    plt.title("Effort versus product size")
  else:
    plt.ylabel("Estimated Time")
    plt.title("Time versus product size")
  plt.show()
  print("\n")

def cocomo(a,b,c,d,kloc,flag,EAF):
  effort = round((a * pow(kloc,b)), 4) * EAF
  time = round((c * pow(effort,d)), 4)
  # personRequired = round((effort / time), 4)
  # productivity = round((kloc / effort),4)
  if flag == 1:
    return effort
  if flag == 2:
    return time
  # print("Effort: " + str(effort)+ " PM")
  # print("Time: " + str(time) + " months")
  # print("Average person required: " + str(personRequired) + " persons")
  # print("Productivity: " + str(productivity) + " KLOC/PM")
  # return effort,
  return effort,time

EAF  = 1
for i in range(0,15):
  
  print("\n")
  print("Very Low- Press 0\nLow - Press 1\nNominal - Press 2\nHigh - Press 3\nVery High - Press 4")
  flag = int(input(informations[i]))
  selectConstants(i,flag)

print("----------------------------------------------------------------------\n")
for i in constants:
      EAF = EAF * i

print("Effort Adjustment Factor: " + str(EAF) + "\n")
effortArr = []
klocArr = []
timeArr = []
for i in range(0,3):
  if(i == 0):
    print("In Organic Mode")
    print("--------------------\n")
    effortArr.append(relay_graph(2.4,1.05,2.5,0.38,kloc, 0,EAF))
    print("\n")
    # timeArr.append(relay_graph_time(2.4,1.05,2.5,0.38,kloc, 0))
  elif(i == 1):
    print("In Semi Detached Mode")
    print("--------------------\n")
    effortArr.append(relay_graph(3.0,1.12,2.5,0.35, kloc, 1,EAF))
    print("\n")
    # timeArr.append(relay_graph_time(3.0	,1.12	,2.5,	0.35, kloc, 1))
  elif(i== 2):
    print("In Embedded Mode")
    print("--------------------\n")
    effortArr.append(relay_graph(3.6,1.20,2.5,0.32, kloc, 2,EAF))
    print("\n")
    # timeArr.append(relay_graph_time(3.6	,1.20,	2.5,	0.32, kloc, 2))
for i in range(1,kloc + 1):
    klocArr.append(i)

# for i in range(0,3):
#   plt.plot(klocArr,effortArr[i], color=colors[i], linewidth=4)
#   i += 1
# draw_curve(1)

# for i in range(0,3):
#   plt.plot(klocArr,timeArr[i], color=colors[i], linewidth=4)
#   i += 1
# draw_curve(0)

Enter total number of lines of code: 
12000


Very Low- Press 0
Low - Press 1
Nominal - Press 2
High - Press 3
Very High - Press 4
Select Required Software Reliability: 2


Very Low- Press 0
Low - Press 1
Nominal - Press 2
High - Press 3
Very High - Press 4
Select Size of Application Database(Select between Low to Very High): 2


Very Low- Press 0
Low - Press 1
Nominal - Press 2
High - Press 3
Very High - Press 4
Select Complexity of The Product: 2


Very Low- Press 0
Low - Press 1
Nominal - Press 2
High - Press 3
Very High - Press 4
Select Runtime Performance Constraints(Select between Nominal to Very High): 2


Very Low- Press 0
Low - Press 1
Nominal - Press 2
High - Press 3
Very High - Press 4
Select Memory Constraints(Select between Nominal to Very High): 2


Very Low- Press 0
Low - Press 1
Nominal - Press 2
High - Press 3
Very High - Press 4
Select Volatility of the virtual machine environment(Select between Low to Very High): 2


Very Low- Press 0
Low - Press 1
Nominal - Press 2
